Conectado a Python 3.11.5

In [1]:
print("Ejecutando archivo entrenamiento")
import torch
import torch.nn as nn
import torchaudio
import torch.fft
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
import seaborn as sns
import pandas as pd
import time
import pickle
import os
import csv
from torch.nn.utils.rnn import pad_sequence
import torchvision.transforms as transforms
import torchaudio.transforms as transformsaudio
import datetime

from torch.utils.tensorboard import SummaryWriter
import soundfile as sf
import time
import torch.nn.functional as F
import torchaudio.functional as Fa

import math
import random

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback

from torchmetrics.audio import PerceptualEvaluationSpeechQuality
from torchmetrics.audio import ShortTimeObjectiveIntelligibility
from torchmetrics.audio import SpeechReverberationModulationEnergyRatio
from torchmetrics.audio import SignalNoiseRatio

import librosa
import tensorflow as tf
import io
from PIL import Image
from prettytable import PrettyTable
import json

import losses
import discriminator
import datasetParaTest
import generator
import wavenet
import postnet

def read_params(params):

    nameOfRun = params["nameOfRun"]
    hardwareAndSize = params["hardwareAndSize"]
    fileLocations= params["fileLocations"]
    noiseConfigs= params["noiseConfigs"]
    wavenetParams = params["wavenetParams"]
    postnetParams = params["postnetParams"]
    trainingHyperParameters = params["trainingHyperParameters"]
    savingAndLogging = params["savingAndLogging"]
    lossesConfig = params["lossesConfig"]
    weightsOfLosses= params["weightsOfLosses"]

    return (nameOfRun, hardwareAndSize , 
            fileLocations, noiseConfigs, 
            wavenetParams, postnetParams, 
            trainingHyperParameters, 
            savingAndLogging, lossesConfig, 
            weightsOfLosses)

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


def generateMelTransform(sample_rate, melspecConfig, device):

    mel_transform = transformsaudio.MelSpectrogram(sample_rate = sample_rate,
                                                    n_fft = melspecConfig["n_fft"],
                                                    n_mels = melspecConfig["n_mels"],
                                                    hop_length = melspecConfig["hop_length"]).to(device)
    
    return mel_transform



def doOneRun(params):


    (nameOfRun, hardwareAndSize , 
            fileLocations, noiseConfigs, 
            wavenetParams, postnetParams, 
            trainingHyperParameters, 
            savingAndLogging, lossesConfig, 
            weightsOfLosses) = read_params(params)
    device = hardwareAndSize["device"]
    sample_rate = hardwareAndSize["sample_rate"]
    inputSize = hardwareAndSize["inputSize"]
    batch_size = hardwareAndSize["batch_size"]
    wavenetOfModel = wavenet.WaveNet(wavenetParams , device)
    postnetOfModel = postnet.PostNetSimple(postnetParams, device)
    discriminatorOfModel = discriminator.Discriminator(device)

    melspec1 = generateMelTransform(sample_rate, lossesConfig["melspec1Config"], device)
    melspec2 = generateMelTransform(sample_rate, lossesConfig["melspec2Config"], device)
    
    nombreParaGuardarLog = nameOfRun#.split("/")
    #nombreParaGuardarLog = nombreParaGuardarLog[-2] + nombreParaGuardarLog[-1] 
    writer = SummaryWriter(log_dir=fileLocations["saveTensorboardLocation"] + "/"+nombreParaGuardarLog , comment=nameOfRun)
    #%load_ext tensorboard
    loss_Fun = losses.CombinedLoss(melspec1, melspec2, device, weightsOfLosses["max_weight_short_pitch"], lossesConfig["epsilon"], lossesConfig["min_db_precision"])

    modeloNombre = nameOfRun+'.pth'
    model = generator.Generator(
                    wavenetOfModel = wavenetOfModel,
                    postnetOfModel = postnetOfModel,
                    discriminatorOfModel = discriminatorOfModel,
                    trainingHyperParameters = trainingHyperParameters,
                    
                    loss_Fun =loss_Fun, device=device, sample_rate =sample_rate,
                    howManyAudiosValidationsSave= savingAndLogging["howManyAudiosValidationsSave"],
                    saveModelIntervalEpochs= savingAndLogging["saveModelIntervalEpochs"], 

                    weightsOfLosses = weightsOfLosses, 
                    mel_transform1=melspec1, mel_transform2=melspec2 ,
                    writer=writer, modeloNombre = modeloNombre)

    file = open("/home/afridman/CorridaFinalTInf.pthpkl",'rb')
    model = pickle.load(file)
    file.close()
   
    #model.load_state_dict(torch.load("/home/afridman/pretrained_weights.pth"))



    print("Importadas todas las librerias y modelo")
    
    
    fullvaldataset = datasetParaTest.AudioCleaningDataset(fileLocations["FullValLocations"], 
                                                   fileLocations["directoryBase"],
                                    target_length=inputSize, noiseConfigs=noiseConfigs, isTrain = False )


    fullValDataloader = DataLoader(fullvaldataset, batch_size=batch_size, shuffle=False, num_workers=16)


    srmrf = SpeechReverberationModulationEnergyRatio(sample_rate).to(device)
    wb_pesqf = PerceptualEvaluationSpeechQuality(sample_rate, 'wb').to(device)
    stoif = ShortTimeObjectiveIntelligibility(sample_rate, False).to(device)
    snrf = SignalNoiseRatio(sample_rate).to(device)

    # Iterate over the entire validation set
    #model.to("cpu")
    generos = ["m","f"]
    paises = ["cl", "pr", "co", "ar", "pe", "ve"]
    tiposCorrida = ["wave", "limpio", "ruido"]
    list_snr, list_pesq, list_stoi, list_srmr, list_loss, generosvistos, paisesvistos, textos = [],[],[],[],[],[],[],[]
    audioidvistos = {}
    numbatches = []
    numuestra = []
    sizes = []
    tardanzas = []
    tiposdecorridalista = []
    reverbNivel = []
    ruidoNivel = []
    ruidoDeTodoslosBatches = noiseConfigs["validationDifficulty"]["snr"]
    reverbDeTodoslosBatches = noiseConfigs["validationDifficulty"]["DificultadReverb"]

    model.eval()
    with torch.no_grad():
        for muestra in range(0,1):
            print(muestra)
            num_batches = 0

            for val_batch in fullValDataloader:
                X, y , audio_file_name, genero, pais, texto= val_batch
                size = (X.shape[2])
                print(size)
                if(size<300000):
                    try:
                        genero = genero[0]
                        pais = pais[0]
                        texto = texto[0]
                        audio_file_name = audio_file_name[0]
                        X = X.to(device)
                        y = y.to(device)
                        ini = time.time()
                        retorno = model(X)
                        fin = time.time()
                        tardanza = fin - ini

                        predichoSoloWave = retorno[0]
                        predichoConPost = retorno[2]

                        sucio = X
                        corridas = [predichoSoloWave,  y, sucio]
                        for i in range(0, len(corridas)):
                            actual = corridas[i]
                            tipo = tiposCorrida[i]

                            lossMel1, lossMel2, customLoss, lossAud , lackAmplitudeLoss, logLoss = loss_Fun(actual, y)
                            lossDiscriminador = 0
                            loss = calculateTotalLoss(weightsOfLosses,lossMel1, lossMel2, customLoss, lossAud , lackAmplitudeLoss, logLoss, lossDiscriminador).to("cpu")

                            snr = snrf(actual,y).to("cpu")#compute_snr(y, y_pred)
                            pesq = wb_pesqf(actual,y).to("cpu")#compute_pesq(y, y_pred)
                            stoi = stoif(actual,y).to("cpu")#compute_stoi(y, y_pred)
                            srmr = srmrf(actual).to("cpu")#compute_srmr(y, y_pred)

                            list_snr.append(float(snr))
                            list_pesq.append(float(pesq))
                            list_stoi.append(float(stoi))
                            list_srmr.append(float(srmr))
                            list_loss.append(float(loss.item()))
                            generosvistos.append(genero)
                            paisesvistos.append(pais)
                            textos.append(texto)
                            numbatches.append(num_batches)
                            numuestra.append(muestra)
                            sizes.append(size)
                            tardanzas.append(tardanza)
                            ruidoNivel.append(ruidoDeTodoslosBatches)
                            reverbNivel.append(reverbDeTodoslosBatches)
                            noiseConfigs["validationDifficulty"]["snr"]
                            tiposdecorridalista.append(tipo)
                            print(num_batches)
                        num_batches += 1
                    except:
                        pass
                    if(num_batches%10==0):
                        datamue = {'snr': list_snr, 'pesq': list_pesq, 
                        'stoi': list_stoi,'srmr': list_srmr,
                        'loss': list_loss, 'textos': textos, 
                        'genero': generosvistos,'pais': paisesvistos, 
                        "muestra": numuestra , "batch":numbatches,
                        "ruido":ruidoNivel,"reverb":reverbNivel,
                        "size": sizes, "tiempoInferencia":tardanzas,
                        "tipo":tiposdecorridalista}
                        df = pd.DataFrame(datamue)
                        df.to_csv("/home/afridman/varianzaResultados"+str(ruidoDeTodoslosBatches)+".csv")
            
        if(num_batches >= 10):
            datamue = {'snr': list_snr, 'pesq': list_pesq, 
            'stoi': list_stoi,'srmr': list_srmr,
            'loss': list_loss, 'textos': textos, 
            'genero': generosvistos,'pais': paisesvistos, 
            "muestra": numuestra , "batch":numbatches,
            "ruido":ruidoNivel,"reverb":reverbNivel,
            "size": sizes, "tiempoInferencia":tardanzas,
              "tipo":tiposdecorridalista}
            df = pd.DataFrame(datamue)
            df.to_csv("/home/afridman/varianzaResultados"+str(ruidoDeTodoslosBatches)+".csv")
            
    #print(audioidvistos)

    """
    with torch.no_grad():
        num_batches = 0
        for val_batch in fullValDataloader:
            X, y , audio_file_name, genero, pais, texto= val_batch
            genero = genero[0]
            pais = pais[0]
            texto = texto[0]
            X = X.to(device)
            y = y.to(device)
            # TODO DESCOMENTAR SI SE ESTA MIRANDO MODELO
            X = model(X)[0]# en vez de 0 poner 2 si tiene post
            print(X.shape)
            
            lossMel1, lossMel2, customLoss, lossAud , lackAmplitudeLoss, logLoss = loss_Fun(X, y)
            lossDiscriminador = 0
            loss = calculateTotalLoss(weightsOfLosses,lossMel1, lossMel2, customLoss, lossAud , lackAmplitudeLoss, logLoss, lossDiscriminador).to("cpu")
            
            # compute metrics
            snr = snrf(X,y).to("cpu")#compute_snr(y, y_pred)
            pesq = wb_pesqf(X,y).to("cpu")#compute_pesq(y, y_pred)
            stoi = stoif(X,y).to("cpu")#compute_stoi(y, y_pred)
            srmr = srmrf(X).to("cpu")#compute_srmr(y, y_pred)

            # Add metrics and loss to the corresponding sum

            
            list_snr.append(float(snr))
            list_pesq.append(float(pesq))
            list_stoi.append(float(stoi))
            list_srmr.append(float(srmr))
            list_loss.append(float(loss.item()))
            generosvistos.append(genero)
            paisesvistos.append(pais)
            textos.append(texto)
            num_batches += 1
    """

    data = {'snr': list_snr, 'pesq': list_pesq, 
            'stoi': list_stoi,'srmr': list_srmr,
             'loss': list_loss, 'textos': textos, 
            'genero': generosvistos,'pais': paisesvistos}#.to("cpu")
    df = pd.DataFrame(data)
    #print(df)
    #df.to_csv("/home/afridman/resultadosTest2.csv")

    

def calculateTotalLoss(weightsOfLosses, lossMel1, lossMel2, customLoss, lossAud , lackAmplitudeLoss, logLoss, lossDiscriminador):
    weightOfMelspecLoss1= weightsOfLosses["weightOfMelspecLoss1"]
    weightOfMelspecLoss2 = weightsOfLosses["weightOfMelspecLoss2"]
    weightOfL1Loss= weightsOfLosses["weightOfL1Loss"]
    weightOfCustomLoss =  weightsOfLosses["weightOfCustomLoss"]
    weightOfAmplitudeLoss = weightsOfLosses["weightOfAmplitudeLoss"]
    weightOfLogLoss = weightsOfLosses["weightOfLogLoss"]
    weightOfDiscriminatorLoss = weightsOfLosses["weightOfDiscriminatorLoss"]
    return lossMel1 * weightOfMelspecLoss1 + lossMel2 * weightOfMelspecLoss2 + lossAud *weightOfL1Loss + customLoss *weightOfCustomLoss + lackAmplitudeLoss * weightOfAmplitudeLoss + logLoss * weightOfLogLoss + lossDiscriminador *weightOfDiscriminatorLoss
    
def saveResults(data, directoryBase):
    # Define the CSV file path

    # Specify the columns
    columns = ['customLoss', 'lackAmplitudeLoss', 'logLoss', 'lossAud', 'lossMel1', 'lossMel2', 'pesq', 'snr', 'srmr', 'stoi', 'val_loss']

    # Specify the file name and path
    file_path = directoryBase + '/results.csv'

    # Check if the file already exists
    if not os.path.isfile(file_path):
        # Create a dataframe with empty values
        df = pd.DataFrame(columns=columns)
        
        # Save the dataframe to a CSV file
        df.to_csv(file_path, index=False)
        print(f"CSV file '{file_path}' created successfully!")
    else:
        print(f"CSV file '{file_path}' already exists.")
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Append a new row to the DataFrame
    df_dictionary = pd.DataFrame([data])
    output = pd.concat([df, df_dictionary], ignore_index=True)
    # Save the updated DataFrame back to the CSV file
    output.to_csv(file_path, index=True)


nombreJson = "/home/afridman/wavenet/paramsRuns/tests.json"

with open(nombreJson, 'r') as file:
    runs = json.load(file)

nameFiles = runs["configRunsName"]
print(nameFiles)

for currentFile in nameFiles:
    currentFile = "/home/afridman/wavenet/paramsRuns/jsonConfigs/" + currentFile
    print(currentFile)
    with open(currentFile, 'r') as file:
        params = json.load(file)
    print("Working on " + currentFile)
    doOneRun(params)

Ejecutando archivo entrenamiento


ImportError: cannot import name 'PerceptualEvaluationSpeechQuality' from 'torchmetrics.audio' (/Users/afridman/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torchmetrics/audio/__init__.py)

In [2]:
pip install torchmetrics.audio

ERROR: Could not find a version that satisfies the requirement torchmetrics.audio (from versions: none)
ERROR: No matching distribution found for torchmetrics.audio

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torchmetrics


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
torchmetrics


NameError: name 'torchmetrics' is not defined

In [5]:
   pip install --upgrade torchmetrics


  Obtaining dependency information for torchmetrics from https://files.pythonhosted.org/packages/a3/88/cc27059747ddecff744826e38014822023cbfff4ca079a6ee9a96602dd0b/torchmetrics-1.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 600.0 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.1.2
    Uninstalling torchmetrics-1.1.2:
      Successfully uninstalled torchmetrics-1.1.2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pesq

  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp311-cp311-macosx_13_0_arm64.whl size=120981 sha256=69c13e57f0d5da829c438272d91da9c453f69fb01aa3a460da938122c44c7c29
  Stored in directory: /Users/afridman/Library/Caches/pip/wheels/ae/f1/23/2698d0bf31eec2b2aa50623b5d93b6206c49c7155d0e31345d
Successfully built pesq
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install stoi

ERROR: Could not find a version that satisfies the requirement stoi (from versions: none)
ERROR: No matching distribution found for stoi
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pystoi

  Preparing metadata (setup.py) ... done
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7778 sha256=81e047fcb77908ae71f57604a16ba2a64408e5a5a60ff6d5e28d76a7b7bda4d9
  Stored in directory: /Users/afridman/Library/Caches/pip/wheels/0f/c6/9a/a456ace9d30e50e6bb37084960037a192f549311c464fa2c86
Successfully built pystoi
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install gammatone

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 179.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 107.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 143.6/208.8 MB 133.6 kB/s eta 0:08:09
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/afridman/.pyenv/versions/3.11.5/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/afridman/.pyenv/versions/3.11.5/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "/Users/afridman/.pyenv/versions/3.11.5/lib/python3.11/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "/Users/afridman/.pyenv/versions/3.11.5/lib/python3.11/site-packages/pip/_vendor/cach

In [13]:
pip istall tensorflow


ERROR: unknown command "istall" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install tensorflow



SyntaxError: invalid syntax (<ipython-input-14-13e4649a079f>, line 1)

In [15]:
pip install tensorflow

  Using cached tensorflow-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.6 kB)
  Using cached tensorflow_macos-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.2 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached wrapt-1.14.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.34.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached tensorboard-2.1